In [1]:
%run common.ipynb

In [2]:
import csv
import urllib

In [3]:
import bs4 as bs

In [4]:
from multiprocessing import Pool


In [5]:
programcode_to_id = pd.read_csv(PROGRAMCODE_TO_ID_FILE)
program_codes = set(programcode_to_id.code)

In [6]:
pd.read_html('http://fgosvo.ru/fgosvo/142/141/16/28', )[0]

,0,1,2
0,Код,Наименование направления,Текст стандарта
1,01.03.01,Математика,Скачать
2,01.03.02,Прикладная математика и информатика,Скачать
3,01.03.03,Механика и математическое моделирование,Скачать
4,01.03.04,Прикладная математика,Скачать


In [7]:
PAT_PROGRAM_LINK = re.compile('^\d{6}.*$')


def extract_program_link(dn):
    if not PAT_PROGRAM_LINK.match(dn.text):
        return
    yield urllib.parse.urljoin('http://fgosvo.ru/', dn.find('a')['href'])

def pagelist():
    pages = bs.BeautifulSoup(requests.get('http://fgosvo.ru/fgosvo/142/141/16').text)
    pages = map(extract_program_link, pages.find_all(class_='dn'))
    pages = itertools.chain.from_iterable(pages)
    return list(pages)
    

program_pages = pagelist()
program_pages

['http://fgosvo.ru/fgosvo/142/141/16/28',
 'http://fgosvo.ru/fgosvo/142/141/16/29',
 'http://fgosvo.ru/fgosvo/142/141/16/3',
 'http://fgosvo.ru/fgosvo/142/141/16/4',
 'http://fgosvo.ru/fgosvo/142/141/16/5',
 'http://fgosvo.ru/fgosvo/142/141/16/6',
 'http://fgosvo.ru/fgosvo/142/141/16/7',
 'http://fgosvo.ru/fgosvo/142/141/16/8',
 'http://fgosvo.ru/fgosvo/142/141/16/9',
 'http://fgosvo.ru/fgosvo/142/141/16/10',
 'http://fgosvo.ru/fgosvo/142/141/16/11',
 'http://fgosvo.ru/fgosvo/142/141/16/12',
 'http://fgosvo.ru/fgosvo/142/141/16/13',
 'http://fgosvo.ru/fgosvo/142/141/16/14',
 'http://fgosvo.ru/fgosvo/142/141/16/15',
 'http://fgosvo.ru/fgosvo/142/141/16/16',
 'http://fgosvo.ru/fgosvo/142/141/16/17',
 'http://fgosvo.ru/fgosvo/142/141/16/18',
 'http://fgosvo.ru/fgosvo/142/141/16/19',
 'http://fgosvo.ru/fgosvo/142/141/16/20',
 'http://fgosvo.ru/fgosvo/142/141/16/21',
 'http://fgosvo.ru/fgosvo/142/141/16/22',
 'http://fgosvo.ru/fgosvo/142/141/16/23',
 'http://fgosvo.ru/fgosvo/142/141/16/24',

In [8]:
PAT_PROGRAMCODE = re.compile(r'\d\d\.\d\d\.\d\d')
assert PAT_PROGRAMCODE.match('01.03.02')
assert not PAT_PROGRAMCODE.match('01,03,02')

In [9]:
def tr_to_record(tr):
    tds = tr.find_all('td')
    if len(tds) != 3:
        return
    program_code = tds[0].text.strip()
    program_code = PAT_PROGRAMCODE.search(program_code)
    if not program_code:
        return
    program_code = program_code.group(0)
    url = tds[2].find('a')['href']
    if not url.endswith('.pdf'):
        return
    url = urllib.parse.urljoin('http://fgosvo.ru/', url)
    yield (program_code, url)
    
def program_url_to_pdflist(url):
    pdfs = requests.get(url).text
    pdfs = bs.BeautifulSoup(pdfs)
    pdfs = pdfs.find('table').find_all('tr')
    pdfs = map(tr_to_record, pdfs)
    return itertools.chain.from_iterable(pdfs)
    

In [10]:
soup010302 = bs.BeautifulSoup(requests.get('http://fgosvo.ru/fgosvo/142/141/16/28').text)

In [11]:
list(itertools.chain.from_iterable(map(tr_to_record, soup010302.find('table').find_all('tr'))))

[('01.03.01',
  'http://fgosvo.ru/fgosvo/downloads/1239/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010301_B_3plus_12122017.pdf'),
 ('01.03.02',
  'http://fgosvo.ru/fgosvo/downloads/1240/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010302_B_3plus_12122017.pdf'),
 ('01.03.03',
  'http://fgosvo.ru/fgosvo/downloads/1241/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010303_B_3plus_12122017.pdf'),
 ('01.03.04',
  'http://fgosvo.ru/fgosvo/downloads/1242/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010304_B_3plus_12122017.pdf')]

In [12]:
list(program_url_to_pdflist('http://fgosvo.ru/fgosvo/142/141/16/3'))

[('03.03.01',
  'http://fgosvo.ru/fgosvo/downloads/1517/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F030301_B_3plus_22112017.pdf'),
 ('03.03.02',
  'http://fgosvo.ru/fgosvo/downloads/1518/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F030302_B_3plus_22112017.pdf'),
 ('03.03.03',
  'http://fgosvo.ru/fgosvo/downloads/1519/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F030303_B_3plus_22112017.pdf')]

* * *

In [13]:
with open('program_pdf_urls.txt', 'w') as f, Pool(5) as pool:
    pdflist = pool.imap(program_url_to_pdflist, program_pages)
    pdflist = itertools.chain.from_iterable(pdflist)
    pdflist = tqdm.tqdm(pdflist)
    
    out = csv.writer(f)
    out.writerow(['code', 'url'])
    i = 0
    def tee(pdf):
        out.writerow(pdf)
        return pdf
    pdflist = map(tee, pdflist)
    pdflist = list(pdflist)

176it [02:10,  1.35it/s]


In [14]:
len(pdflist)

176

In [15]:
len(program_codes)

459

In [16]:
! wc -l programs.csv

291 programs.csv


In [17]:
pdflist

[('01.03.01',
  'http://fgosvo.ru/fgosvo/downloads/1239/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010301_B_3plus_12122017.pdf'),
 ('01.03.02',
  'http://fgosvo.ru/fgosvo/downloads/1240/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010302_B_3plus_12122017.pdf'),
 ('01.03.03',
  'http://fgosvo.ru/fgosvo/downloads/1241/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010303_B_3plus_12122017.pdf'),
 ('01.03.04',
  'http://fgosvo.ru/fgosvo/downloads/1242/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F010304_B_3plus_12122017.pdf'),
 ('02.03.01',
  'http://fgosvo.ru/fgosvo/downloads/991/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F020301_B_3plus_21062017.pdf'),
 ('02.03.02',
  'http://fgosvo.ru/fgosvo/downloads/992/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F020302_B_3plus_21062017.pdf'),
 ('02.03.03',
  'http://fgosvo.ru/fgosvo/downloads/993/?f=%2Fuploadfiles%2FProjFGOSVO3%2B%2B%2FBak3%2B%2B%2F020303_B_3plus_21062017.pdf'),
 ('03.03.01',
  'http:/

**Long story short**, some of the educational programs that I've parsed from minobrnauki reports are simply undocumented...

In [18]:
list(program_codes.difference([code for code, url in pdflist]))[:5]

['11.04.04', '12.05.01', '04.04.02', '41.04.05', '24.04.01']

In [19]:
%%time

def dl_pdf(code_and_url):
    code, url = code_and_url
    dst_path = os.path.join('programs', code + '.pdf')
    if not os.path.exists('programs'):
        os.mkdir('programs')
    with open(dst_path, 'wb') as out, requests.get(url, stream=True) as rsp:
        out.write(b'')
        for chunk in rsp.iter_content(1024):
            out.write(chunk)
    return dst_path

with Pool(5) as pool:
    downloaded = pool.imap(dl_pdf, pdflist)
    for d in tqdm.tqdm(downloaded):
        print('Finished downloading {}'.format(d))

1it [00:04,  4.16s/it]

Finished downloading programs/01.03.01.pdf
Finished downloading programs/01.03.02.pdf
Finished downloading programs/01.03.03.pdf
Finished downloading programs/01.03.04.pdf


5it [00:05,  3.04s/it]

Finished downloading programs/02.03.01.pdf


6it [00:06,  2.20s/it]

...

Finished downloading programs/09.03.03.pdf
Finished downloading programs/09.03.04.pdf
Finished downloading programs/10.03.01.pdf
Finished downloading programs/11.03.01.pdf


32it [02:29, 10.71s/it]

Finished downloading programs/11.03.02.pdf
Finished downloading programs/11.03.03.pdf
Finished downloading programs/11.03.04.pdf
Finished downloading programs/12.03.01.pdf
Finished downloading programs/12.03.02.pdf
Finished downloading programs/12.03.03.pdf
Finished downloading programs/12.03.04.pdf
Finished downloading programs/12.03.05.pdf
Finished downloading programs/13.03.01.pdf
Finished downloading programs/13.03.02.pdf
Finished downloading programs/13.03.03.pdf
Finished downloading programs/14.03.01.pdf
Finished downloading programs/14.03.02.pdf
Finished downloading programs/15.03.01.pdf
Finished downloading programs/15.03.02.pdf
Finished downloading programs/15.03.03.pdf
Finished downloading programs/15.03.04.pdf
Finished downloading programs/15.03.05.pdf
Finished downloading programs/15.03.06.pdf
Finished downloading programs/16.03.01.pdf
Finished downloading programs/16.03.02.pdf
Finished downloading programs/16.03.03.pdf
Finished downloading programs/18.03.01.pdf
Finished do

176it [03:47,  1.29s/it]

Finished downloading programs/45.03.03.pdf
Finished downloading programs/45.03.04.pdf
Finished downloading programs/46.03.04.pdf
Finished downloading programs/47.03.01.pdf
Finished downloading programs/47.03.02.pdf
Finished downloading programs/47.03.03.pdf
Finished downloading programs/49.03.01.pdf
Finished downloading programs/49.03.02.pdf
Finished downloading programs/49.03.03.pdf
Finished downloading programs/50.03.01.pdf
Finished downloading programs/50.03.02.pdf
Finished downloading programs/50.03.03.pdf
Finished downloading programs/50.03.04.pdf
Finished downloading programs/51.03.01.pdf
Finished downloading programs/51.03.02.pdf
Finished downloading programs/51.03.03.pdf
Finished downloading programs/51.03.04.pdf
Finished downloading programs/51.03.05.pdf
Finished downloading programs/51.03.06.pdf
Finished downloading programs/52.03.01.pdf
Finished downloading programs/52.03.02.pdf
Finished downloading programs/52.03.03.pdf
Finished downloading programs/52.03.04.pdf
Finished do

Not going to validate, will assume it's all ok

* * *

There are some problems with running `ghostscript` in conda/jupyter under firejail and `camelot-py` (which I'm using to extract tables) heavily depends on it, so there's `extract_fgosvo.py` in the project root which can be run outside conda enviroment. It's got a number of rather shameful hacks, takes a while to run, and needs increased `ulimit -n`.